This jupyter notebook is licensed under Creative Commons 2.0 By Attribution  
License information is here: [https://creativecommons.org/licenses/by/2.0/](https://creativecommons.org/licenses/by/2.0/)  
Author: Lauren Oldja, [laurenoldja.net](http://laurenoldja.net)

In [1]:
import pandas as pd
import numpy as np
import os

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Data source: http://web.mta.info/developers/turnstile.html
#Download all desired weeks from website, store them in a folder called ../data/mta_turnstiles
# This will load combined CSV files that are in this folder into an appended dataframe
datafiles = ['data/mta_turnstiles/' + x for x in os.listdir('data/mta_turnstiles/')]

list_ = []
for file_ in datafiles:
    df = pd.read_csv(file_)
    list_.append(df)
df = pd.concat(list_)

In [3]:
df.columns #notice the whitespace on EXITS

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES',
       u'EXITS                                                               '],
      dtype='object')

In [4]:
df.columns = df.columns.str.strip() #strip whitespace
df.head() #preview first five rows

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,00:00:00,REGULAR,5672118,1920707
1,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,04:00:00,REGULAR,5672183,1920719
2,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,08:00:00,REGULAR,5672214,1920761
3,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,12:00:00,REGULAR,5672330,1920867
4,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,16:00:00,REGULAR,5672640,1920936


In [5]:
df.describe() #basic descriptive statistics

,ENTRIES,EXITS
count,7.772530e+05,7.772530e+05
mean,3.619627e+07,2.946001e+07
std,1.975870e+08,1.791294e+08
min,0.000000e+00,0.000000e+00
25%,5.873780e+05,2.979540e+05
50%,2.575607e+06,1.501834e+06
75%,6.546763e+06,4.657076e+06
max,2.147483e+09,2.087387e+09


#  Basic Cleaning

## Drop duplicates

In [6]:
df.duplicated().value_counts()

False    777253
dtype: int64

In [7]:
df = df.drop_duplicates()

## Make a datetime obj timestamp

In [8]:
df['TIMESTAMP'] = pd.to_datetime((df.DATE + ' ' + df.TIME), format='%m/%d/%Y %H:%M:%S')

Create a column with the day of the week this timestamp shows:

In [9]:
import datetime as dt

weekdays = ['MON','TUE','WED','THU','FRI','SAT','SUN']

df['DATE'][1]
df['DOF'] = [weekdays[dt.datetime.strptime(dstring,'%m/%d/%Y').weekday()] for dstring in df.DATE.tolist()]
# DOF = "day of week"

In [10]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TIMESTAMP,DOF
0,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,00:00:00,REGULAR,5672118,1920707,2016-05-21 00:00:00,SAT
1,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,04:00:00,REGULAR,5672183,1920719,2016-05-21 04:00:00,SAT
2,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,08:00:00,REGULAR,5672214,1920761,2016-05-21 08:00:00,SAT
3,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,12:00:00,REGULAR,5672330,1920867,2016-05-21 12:00:00,SAT
4,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,16:00:00,REGULAR,5672640,1920936,2016-05-21 16:00:00,SAT


## Make unique identifiers for stations

In [11]:
df = df.reset_index()

In [12]:
l = [''.join(sorted(a)) for a in df['LINENAME']] #sort each linename, since subway lines aren't listed in a consistent order

In [13]:
df['STATID']=df['STATION']+pd.Series(l)

## Turnstiles capture cumulative counts, but we want noncumulative counts
Get the row difference in order to get a count per time period. Assign this a new column.

In [14]:
df['ENTRY_DIFF']=df.groupby(['STATID','UNIT','SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

Let's see some summary statistics on ENTRY_DIFF by station to identify outliers.

In [15]:
max_table = df.sort_values(['ENTRY_DIFF']).groupby(['STATID'])['STATID','ENTRY_DIFF'].max()
# print max_table

Yikes some are really big. Let's see the really big ones only.

In [33]:
max_table[max_table['ENTRY_DIFF'] > 100000 ]

,STATID,ENTRY_DIFF
STATID,,
33 ST6,33 ST6,1.162241e+08
34 ST-PENN STAACE,34 ST-PENN STAACE,2.402797e+06
51 ST6,51 ST6,1.403818e+09
AVENUE HBQ,AVENUE HBQ,1.174165e+08
CANAL ST6JNQRZ,CANAL ST6JNQRZ,3.175049e+08
CANARSIE-ROCKAWL,CANARSIE-ROCKAWL,4.951429e+07
FULTON ST2345ACJZ,FULTON ST2345ACJZ,1.174397e+08
GRD CNTRL-42 ST4567S,GRD CNTRL-42 ST4567S,1.879008e+09
JAY ST-METROTECACF,JAY ST-METROTECACF,2.076115e+09


In [17]:
min_table = df.sort_values(['ENTRY_DIFF']).groupby(['STATID'])['STATID','ENTRY_DIFF'].min()
# print min_table

In [18]:
min_table[min_table['ENTRY_DIFF'] < 0 ] #show negative values

,STATID,ENTRY_DIFF
STATID,,
1 AVL,1 AVL,-1.510852e+08
104 STA,104 STA,-1.250000e+02
104 STJZ,104 STJZ,-1.970000e+02
125 ST23,125 ST23,-9.950000e+02
125 ST456,125 ST456,-2.682000e+03
14 ST123FLM,14 ST123FLM,-1.143000e+03
14TH STREET1,14TH STREET1,-2.728372e+06
161/YANKEE STAD4BD,161/YANKEE STAD4BD,-8.280000e+02
170 STBD,170 STBD,-5.578590e+05


We've got some negative numbers here. The counters may be running backwards. We should drill down further to investigate, and explicitly throw out any outliers that are clearly indicative of mechanical malfunction (i.e. vary large or very negative numbers in low traffic areas). Let's go back and take the absolute values before we take row differences.  

Strange numbers may also indicate that a counter has been reset. It's worth drilling down for a closer look at the data in this case.

In [19]:
df['ENTRY_DIFF_ABS']=df['ENTRY_DIFF'].abs()
min_table = df.sort_values(['ENTRY_DIFF_ABS']).groupby(['STATID'])['STATID','ENTRY_DIFF_ABS'].min()

In [20]:
min_table

,STATID,ENTRY_DIFF_ABS
STATID,,
1 AVL,1 AVL,0.0
103 ST-CORONA7,103 ST-CORONA7,0.0
103 ST1,103 ST1,3.0
103 ST6,103 ST6,0.0
103 STBC,103 STBC,0.0
104 STA,104 STA,0.0
104 STJZ,104 STJZ,0.0
110 ST6,110 ST6,0.0
111 ST7,111 ST7,6.0


So many zeros! It's possible that this is true for some stations at some time periods. But in some cases the turnstile is probably just broken. Let's keep going. We can throw out the zeros after we bin the hourly data (if we do this before, it will be harder to tell if our bins are uniformly sized)

## Bin timestamps by hour
We have data roughly at four-hour intervals, but it is not consistent within stations or across lines. Let's bin it.

In [21]:
bins = [-1,3,7,11,15,19,24] #use a negative number at the beginning to ensure we do not lose midnight

In [22]:
df['HOD'] = [r.hour for r in df.TIMESTAMP] #hod = "hour of day"
df['HODBIN'] = pd.cut(df['HOD'], bins)

In [23]:
df.HODBIN.value_counts()

(7, 11]     133285
(3, 7]      131890
(11, 15]    128746
(15, 19]    128215
(-1, 3]     127613
(19, 24]    127504
dtype: int64

In [24]:
df.groupby(['STATID','HODBIN']).sum()

index      ENTRIES        EXITS  \
STATID               HODBIN                                         
1 AVL                (-1, 3]    8381570  41308618010  65131878470   
                     (3, 7]     8681394  42195326126  66003223660   
                     (7, 11]    8828993  42638507564  66438811853   
                     (11, 15]   8560810  41903090807  67580385111   
                     (15, 19]   8412575  41308949092  65132180874   
                     (19, 24]   8084542  39747479552  61656053968   
103 ST-CORONA7       (-1, 3]   45125983   1785916471   1702408993   
                     (3, 7]    45126235   1785923974   1702439760   
                     (7, 11]   44947308   1777320587   1695739995   
                     (11, 15]  45126738   1786226043   1702497260   
                     (15, 19]  45103617   1786481405   1702692239   
                     (19, 24]  45127242   1786388335   1702703066   
103 ST1              (-1, 3]   21042954   1702964768    871748416   
                     (3, 7]    21848607   1768437200    905267087   
                     (7, 11]   21043272   1703032641    871765592   
                     (11, 15]  20228109   1637728544    838313630   
                     (15, 19]  20228259   1637796750    838346681   
                     (19, 24]  20228409   1637863343    838391504   
103 ST6              (-1, 3]   25966765   1753981845  15082734333   
                     (3, 7]    25966932   1753985288  15082741375   
                     (7, 11]   26907512   1816980896  15621516027   
                     (11, 15]  26124009   1766177521  15088077827   
                     (15, 19]  26124177   1766272954  15088149587   
                     (19, 24]  25967608   1754306974  15082994684   
103 STBC             (-1, 3]    3815976    709745836    633326492   
                     (3, 7]     3816060    709746768    633328582   
                     (7, 11]    3952671    735149357    655972599   
                     (11, 15]   3816231    709806180    633348463   
                     (15, 19]   3816315    709832259    633369157   
                     (19, 24]   3816399    709856622    633398335   
...                                 ...          ...          ...   
WOODHAVEN BLVDMR     (-1, 3]   25143221   1615829355   1063534905   
                     (3, 7]    25471015   1627012660   1067224404   
                     (7, 11]   29245406   1916842428   1276081883   
                     (11, 15]  25637665   1662522204   1097793371   
                     (15, 19]  25715811   1662971325   1098147214   
                     (19, 24]  25144828   1616358010   1063841629   
WOODLAWN4            (-1, 3]   22436680    784752963    319555192   
                     (3, 7]    22436820    784756979    319558546   
                     (7, 11]   22436960    784811859    319562022   
                     (11, 15]  22437100    784851887    319568799   
                     (15, 19]  22437240    784888779    319578254   
                     (19, 24]  22437380    784927575    319595235   
WORLD TRADE CTR23ACE (-1, 3]   17617978   1049798814    545443315   
                     (3, 7]    17618286   1049803339    545446288   
                     (7, 11]   17561066   1041621847    539520377   
                     (11, 15]  17618901   1049868694    545526385   
                     (15, 19]  17619209   1049961674    545566378   
                     (19, 24]  17619517   1050130931    545614532   
YORK STF             (-1, 3]    8375103    912961711    829515373   
                     (3, 7]     8375187    912963567    829517443   
                     (7, 11]    8680941    945313722    858976819   
                     (11, 15]   8375358    913031304    829603479   
                     (15, 19]   8375442    913099684    829641465   
                     (19, 24]   8375526    913196063    829677925   
ZEREGA AV6           (-1, 3]   28638414   1506446679     21451497   
                     (3, 7]    2

Focus on ENTRY_DIFF_ABS, none of the other sums mean anything. These bins look good.

# Data exploration
Evaluate max entries to identify possible outliers.

In [25]:
df.sort_values(['ENTRY_DIFF_ABS']).groupby(['STATID'])['STATID','ENTRY_DIFF_ABS'].max()

,STATID,ENTRY_DIFF_ABS
STATID,,
1 AVL,1 AVL,151085187.0
103 ST-CORONA7,103 ST-CORONA7,2027.0
103 ST1,103 ST1,9003.0
103 ST6,103 ST6,1301.0
103 STBC,103 STBC,841.0
104 STA,104 STA,355.0
104 STJZ,104 STJZ,431.0
110 ST6,110 ST6,1272.0
111 ST7,111 ST7,1242.0


It looks like we might have some crazy outliers. What is going on with Wall St45? Let's take a look.

In [26]:
df[df['STATID'] == "WALL ST45"]

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TIMESTAMP,DOF,STATID,ENTRY_DIFF,ENTRY_DIFF_ABS,HOD,HODBIN
141380,141380,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,02:00:00,REGULAR,1753042,735701,2016-05-21 02:00:00,SAT,WALL ST45,NaN,NaN,2,"(-1, 3]"
141381,141381,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,06:00:00,REGULAR,1753042,735702,2016-05-21 06:00:00,SAT,WALL ST45,0.0,0.0,6,"(3, 7]"
141382,141382,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,07:58:21,REGULAR,1753053,735702,2016-05-21 07:58:21,SAT,WALL ST45,11.0,11.0,7,"(3, 7]"
141383,141383,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,07:59:59,REGULAR,1753053,735702,2016-05-21 07:59:59,SAT,WALL ST45,0.0,0.0,7,"(3, 7]"
141384,141384,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,10:00:00,REGULAR,1753077,735703,2016-05-21 10:00:00,SAT,WALL ST45,24.0,24.0,10,"(7, 11]"
141385,141385,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,14:00:00,REGULAR,1753204,735712,2016-05-21 14:00:00,SAT,WALL ST45,127.0,127.0,14,"(11, 15]"
141386,141386,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,18:00:00,REGULAR,1753381,735719,2016-05-21 18:00:00,SAT,WALL ST45,177.0,177.0,18,"(15, 19]"
141387,141387,R203,R043,00-00-00,WALL ST,45,IRT,05/21/2016,22:00:00,REGULAR,1753485,735722,2016-05-21 22:00:00,SAT,WALL ST45,104.0,104.0,22,"(19, 24]"
141388,141388,R203,R043,00-00-00,WALL ST,45,IRT,05/22/2016,02:00:00,REGULAR,1753505,735722,2016-05-22 02:00:00,SUN,WALL ST45,20.0,20.0,2,"(-1, 3]"
141389,141389,R203,R043,00-00-00,WALL ST,45,IRT,05/22/2016,06:00:00,REGULAR,1753507,735722,2016-05-22 06:00:00,SUN,WALL ST45,2.0,2.0,6,"(3, 7]"


In [27]:
df[df['STATID'] == "WALL ST23"]

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TIMESTAMP,DOF,STATID,ENTRY_DIFF,ENTRY_DIFF_ABS,HOD,HODBIN
121389,121389,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,00:00:00,REGULAR,3648884,5424996,2016-05-21 00:00:00,SAT,WALL ST23,NaN,NaN,0,"(-1, 3]"
121390,121390,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,04:00:00,REGULAR,3648896,5425018,2016-05-21 04:00:00,SAT,WALL ST23,12.0,12.0,4,"(3, 7]"
121391,121391,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,08:00:00,REGULAR,3648918,5425060,2016-05-21 08:00:00,SAT,WALL ST23,22.0,22.0,8,"(7, 11]"
121392,121392,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,08:21:11,REGULAR,3648919,5425073,2016-05-21 08:21:11,SAT,WALL ST23,1.0,1.0,8,"(7, 11]"
121393,121393,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,12:00:00,REGULAR,3648971,5425251,2016-05-21 12:00:00,SAT,WALL ST23,52.0,52.0,12,"(11, 15]"
121394,121394,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,16:00:00,REGULAR,3649078,5425440,2016-05-21 16:00:00,SAT,WALL ST23,107.0,107.0,16,"(15, 19]"
121395,121395,R110,R027,01-00-00,WALL ST,23,IRT,05/21/2016,20:00:00,REGULAR,3649203,5425565,2016-05-21 20:00:00,SAT,WALL ST23,125.0,125.0,20,"(19, 24]"
121396,121396,R110,R027,01-00-00,WALL ST,23,IRT,05/22/2016,00:00:00,REGULAR,3649234,5425623,2016-05-22 00:00:00,SUN,WALL ST23,31.0,31.0,0,"(-1, 3]"
121397,121397,R110,R027,01-00-00,WALL ST,23,IRT,05/22/2016,04:00:00,REGULAR,3649240,5425650,2016-05-22 04:00:00,SUN,WALL ST23,6.0,6.0,4,"(3, 7]"
121398,121398,R110,R027,01-00-00,WALL ST,23,IRT,05/22/2016,08:00:00,REGULAR,3649241,5425676,2016-05-22 08:00:00,SUN,WALL ST23,1.0,1.0,8,"(7, 11]"


It looks like we have 1000 more rows for Wall St 23 than for Wall St 45, and that might account for some of the craziness. By taking a close look at HOD, we see some instances of two measurements taken within the same bin. But multiple measurements for the same time period should just result in small counts at each time. Let's take another approach.

In [28]:
df.sort_values(['ENTRY_DIFF_ABS']).groupby(['STATID','DOF','HODBIN'])['STATID','ENTRY_DIFF_ABS'].sum()

ENTRY_DIFF_ABS
STATID     DOF HODBIN                  
1 AVL      FRI (-1, 3]          18438.0
               (3, 7]            4000.0
               (7, 11]           6731.0
               (11, 15]         21397.0
               (15, 19]         22171.0
               (19, 24]         22392.0
           MON (-1, 3]          11410.0
               (3, 7]            2573.0
               (7, 11]           6151.0
               (11, 15]         17640.0
               (15, 19]         18313.0
               (19, 24]         24814.0
           SAT (-1, 3]          21448.0
               (3, 7]            7667.0
               (7, 11]           2912.0
               (11, 15]         11277.0
               (15, 19]         19249.0
               (19, 24]         19732.0
           SUN (-1, 3]          16433.0
               (3, 7]            6680.0
               (7, 11]           2072.0
               (11, 15]          8610.0
               (15, 19]         15583.0
               (19, 24]         16248.0
           THU (-1, 3]          17612.0
               (3, 7]            2951.0
               (7, 11]           7410.0
               (11, 15]         22647.0
               (15, 19]     151104944.0
               (19, 24]         29447.0
...                                 ...
ZEREGA AV6 SAT (-1, 3]            280.0
               (3, 7]             101.0
               (7, 11]           1222.0
               (11, 15]          1373.0
               (15, 19]          1412.0
               (19, 24]           800.0
           SUN (-1, 3]            406.0
               (3, 7]             104.0
               (7, 11]            732.0
               (11, 15]          1200.0
               (15, 19]          1041.0
               (19, 24]           634.0
           THU (-1, 3]            371.0
               (3, 7]             273.0
               (7, 11]           4665.0
               (11, 15]          2041.0
               (15, 19]          2357.0
               (19, 24]          1373.0
           TUE (-1, 3]            308.0
               (3, 7]             229.0
               (7, 11]           4743.0
               (11, 15]          1998.0
               (15, 19]          2339.0
               (19, 24]          1336.0
           WED (-1, 3]            374.0
               (3, 7]             249.0
               (7, 11]           4925.0
               (11, 15]          2003.0
               (15, 19]          2277.0
               (19, 24]          1510.0

[19404 rows x 1 columns]

# Let's make some quick plots

Look for this in Part Two of the tutorial!

If you liked this jupyter notebook or have any comments/questions/suggestions, please email me via the form at [laurenoldja.net/contact](laurenoldja.net/contact)